In [1]:
import spacy
import pickle
import random

First, we will execute spacy model on a simple sentence just to get familiar with it.

In [3]:
test = spacy.load('en')
sent = '''My name is P Kumar Deb, i stay in Jaipur.
The 2023 ICC Cricket World Cup is scheduled for October.'''

ts = test(sent)
for ent in ts.ents:
  print(f'{ent.label_.upper():{10}} - {ent.text}')


ORG        - P Kumar Deb
GPE        - Jaipur
DATE       - 2023
EVENT      - World Cup
DATE       - October


In [6]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dataset-for-resume-information-retrieval/Alice Clark CV.pdf
/kaggle/input/dataset-for-resume-information-retrieval/train_data.pkl


In [7]:
pip install PyMuPDF 

     |████████████████████████████████| 14.1 MB 4.3 MB/s eta 0:00:01    |███▊                            | 1.7 MB 4.3 MB/s eta 0:00:03
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import sys
import fitz

fname = '/kaggle/input/dataset-for-resume-information-retrieval/Alice Clark CV.pdf'
doc = fitz.open(fname)
alice_cv = ""

for page in doc:
    alice_cv += page.get_text()

print(alice_cv)


Alice Clark 
AI / Machine Learning 
 
Delhi, India Email me on Indeed 
• 
20+ years of experience in data handling, design, and development 
• 
Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to 
data warehousing and business intelligence 
• 
Database: Experience in database designing, scalability, back-up and recovery, writing and 
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. 
Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, 
Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake 
analytics(U-SQL) 
Willing to relocate anywhere 
 
WORK EXPERIENCE 
Software Engineer 
Microsoft – Bangalore, Karnataka 
January 2000 to Present 
1. Microsoft Rewards Live dashboards: 
Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping 
online. Microsoft Rewards members can earn points when searching with Bing, bro

In [24]:
fname = '/kaggle/input/resume/resume3.pdf'
doc = fitz.open(fname)
resume_3 = ""

for page in doc:
    resume_3 += page.get_text()

print(resume_3)


Nivetha B 
Looking for an exciting role as a student intern to utilize my development skills for the growth of the organization
as well as to enhance my knowledge. 
nivethabalaji1312@gmail.com 
8925706279 
Chennai, Tamilnadu. 
linkedin.com/in/nivetha-balaji 
github.com/Nivetha1312 
EDUCATION 
MSc Data Science 
PSG College of Technology 
2019 - 2024,  
8.07 CGPA 
XII(Higher Secondary, State Board) 
Bharathidhasanar Matriculation Higher Secondary
School, Arakkonam. 
2018 - 2019,  
91.8% 
X(SSLC, State Board) 
Bharathidhasanar Matriculation Higher Secondary
School, Arakkonam. 
2016 - 2017,  
96.6% 
PROJECTS 
Imagado 
An application developed using Python, trained using the dataset
comprising 30 pictures each of six famous personalities. The purpose of
this project is to achieve student attendance marking, which is capable
of identifying a person from their digital image. This task is
accomplished using Logistic Regression due to greater accuracy. 
Time for the Trend 
A project developed i

Now, we can pass our test data on our pre trained spacy model and evaluate how good it has performed.

In [10]:
test = spacy.load('en')
ts = test(" ".join(alice_cv.split('\n'))) # we have splitted our data with '\n' and rejoined with space. 

**ts** variable contains all POS tags,NER etc in it. We will extract only NER and manually verify output.

In [11]:
for ent in ts.ents:
  if ent.label_.upper() == 'PERSON':
    print(f'{ent.label_.upper():{10}} - {ent.text}')

PERSON     - Alice Clark
PERSON     - Stored Procedures
PERSON     - Cloud
PERSON     - Document DB
PERSON     - Web Job
PERSON     - Web App
PERSON     - Bing


In [12]:
for ent in ts.ents:
  if ent.label_.upper() == 'ORG':
    print(f'{ent.label_.upper():{10}} - {ent.text}')

ORG        - AI / Machine Learning
ORG        - star/snow flake scema
ORG        - SQL
ORG        - Microsoft Azure
ORG        - SQL Azure
ORG        - Stream Analytics
ORG        - Power BI
ORG        - Power BI
ORG        - SQL
ORG        - Microsoft
ORG        - Microsoft
ORG        - Microsoft
ORG        - Microsoft
ORG        - Microsoft Edge
ORG        - Microsoft
ORG        - Microsoft
ORG        - PBI
ORG        - Technology/Tools
ORG        - Indian Institute of Technology
ORG        - Big Data


In [13]:
train_data = pickle.load(open('/kaggle/input/dataset-for-resume-information-retrieval/train_data.pkl','rb'))
print(f"Training data consist of {len(train_data)} manually labelled resume's.")

Training data consist of 200 manually labelled resume's.


In [14]:
train_data[97]

('Ramesh chokkala Telangana - Email me on Indeed: indeed.com/r/Ramesh-chokkala/16d5fa56f8c19eb6  WORK EXPERIENCE  software  Microsoft,Infosis, Google -  May 2018 to Present  software  Microsoft,Infosis, Google -  May 2018 to Present  EDUCATION  btech  Trinity engineering college  https://www.indeed.com/r/Ramesh-chokkala/16d5fa56f8c19eb6?isid=rex-download&ikw=download-top&co=IN',
 {'entities': [(250, 278, 'College Name'),
   (243, 248, 'Degree'),
   (182, 207, 'Companies worked at'),
   (172, 180, 'Designation'),
   (122, 147, 'Companies worked at'),
   (112, 120, 'Designation'),
   (48, 94, 'Email Address'),
   (16, 25, 'Location'),
   (0, 15, 'Name')]})

In [15]:
# loading blank spacy model as we want to customize our model.
# spacy.blank('en') will create a blank model of a given language class i.e., for here English.

nlp = spacy.blank('en') 

In [16]:
# Creating a function to train our model

def train_model(train_data):
    
  if 'ner' not in nlp.pipe_names:# Checking if NER is present in pipeline
    ner = nlp.create_pipe('ner') # creating NER pipe if not present
    nlp.add_pipe(ner, last=True) # adding NER pipe in the end

  for _, annotation in train_data: # Getting 1 resume at a time from our training data of 200 resumes
    for ent in annotation['entities']: # Getting each tuple at a time from 'entities' key in dictionary at index[1] i.e.,(0, 15, 'Name') and so on
      ner.add_label(ent[2])  # here we are adding only labels of each tuple from entities key dict, eg:- 'Name' label of (0, 15, 'Name')
    
  # In above for loop we finally added all custom NER from training data.
    

  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner'] # getting all other pipes except NER.
  with nlp.disable_pipes(*other_pipes): # Disabling other pipe's as we want to train only NER.
        optimizer = nlp.begin_training()
        
        for itn in range(10):         # trainig model for 10 iteraion
            print('Starting iteration ' + str(itn))
            random.shuffle(train_data) # shuffling data in every iteration 
            losses = {}
            for text, annotations in train_data:
              try:
                nlp.update(
                    [text],        #batch of texts
                    [annotations], #batch of annotations
                    drop=0.2,      #dropout rate -makes it harder to memorise
                    sgd=optimizer, #callable to update weights
                    losses=losses) #Dictionary to update with the loss, keyed by pipeline component.
              except Exception as e:
                pass

In [17]:
train_model(train_data)

Starting iteration 0
Starting iteration 1
Starting iteration 2
Starting iteration 3
Starting iteration 4
Starting iteration 5
Starting iteration 6
Starting iteration 7
Starting iteration 8
Starting iteration 9


In [18]:
nlp.to_disk('nlp_model')

In [19]:
nlp_model = spacy.load('nlp_model')

In [20]:
nlp_model.entity.labels

('College Name',
 'Companies worked at',
 'Degree',
 'Designation',
 'Email Address',
 'Graduation Year',
 'Location',
 'Name',
 'Skills',
 'UNKNOWN',
 'Years of Experience')

In [22]:
doc = nlp_model(" ".join(alice_cv.split('\n')))
for ent in doc.ents:
  print(f'{ent.label_.upper():{20}} - {ent.text}')

NAME                 - Alice Clark
LOCATION             - Delhi
COMPANIES WORKED AT  - Microsoft Azure
DESIGNATION          - Software Engineer
COMPANIES WORKED AT  - Microsoft
COMPANIES WORKED AT  - Microsoft
COMPANIES WORKED AT  - Microsoft
COMPANIES WORKED AT  - Microsoft
COMPANIES WORKED AT  - Microsoft
COMPANIES WORKED AT  - Microsoft
COMPANIES WORKED AT  - Microsoft
COLLEGE NAME         - Indian Institute of Technology – Mumbai
SKILLS               - Machine Learning, Natural Language Processing, and Big Data Handling    ADDITIONAL INFORMATION  Professional Skills  • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal


From above output we can clearly see that our custom trained spacy model has worked very well and labelled our testing data correctly but not 100% as the skills are not mentioned in output. This could be due to less training data. To increase accuracy we should train our model on different formats of data.

In [25]:
doc = nlp_model(" ".join(resume_3.split('\n')))
for ent in doc.ents:
  print(f'{ent.label_.upper():{20}} - {ent.text}')

NAME                 - Nivetha B
DEGREE               - MSc Data Science
COMPANIES WORKED AT  - PSG College
DEGREE               - Python
COMPANIES WORKED AT  - PSG College
